In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [3]:
#move to path
%cd /content/drive/MyDrive/Omok/

/content/drive/MyDrive/Omok


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
#데이터 불러오고, 학습 데이터셋 생성

w, h = 15, 15
base_path = os.path.join('preprocessed_Data', '*.npz')

file_list = glob(base_path)

x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

print(f"x_data length: {len(x_data)}, y_data length: {len(y_data)}")
print(f"First element shape in x_data: {x_data[0].shape}, First element shape in y_data: {y_data[0]}")

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.15, random_state=2024)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
#모델 구조 정의
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Flatten(),  # Flatten 레이어 추가
    layers.Dense(h * w, activation='sigmoid')  # 타겟 값의 형태에 맞추어 출력
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

'''
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.Dropout(0.5),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Flatten(),  # Flatten 레이어 추가
    layers.Dense(h * w, activation='sigmoid')  # 타겟 값의 형태에 맞추어 출력
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()
'''

In [ ]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)
print(x_train.shape)
print(y_train.shape)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.keras' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

**추가 학습 (optional)**

In [ ]:
from tensorflow.keras.models import load_model

# 기존에 학습된 모델 불러오기
model = load_model('model.keras')

print(len(model.layers))

# 필요한 레이어만 재학습에 사용
for i, layer in enumerate(model.layers):
    if i < 5 or i >= len(model.layers) - 8:
        layer.trainable = False
    else:
        layer.trainable = True

model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    callbacks=[
        ModelCheckpoint('./models/%s.keras' % ("added_Model"), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

**keras 모델 파일을 json과 h5형식으로 변환**

모델 구조와 가중치를 따로 저장해서 다른 플랫폼에서도 모델을 오류 없이 불러올 수 있도록 함.

In [ ]:
from tensorflow.keras.models import load_model, model_from_json

# 기존 모델 로드 (에러 발생 가능성 있음)
try:
    model = load_model('./model.keras')
except Exception as e:
    print("Error loading model:", e)
    # 모델이 로드되지 않으면 종료
    raise e

# 모델 구조를 JSON 형식으로 저장
model_json = model.to_json()
with open("./model_structure.json", "w") as json_file:
    json_file.write(model_json)

# 모델 가중치를 HDF5 형식으로 저장
model.save_weights("./model_weight.h5")
print("Model saved in JSON and HDF5 formats.")